In [1]:
from tinygrad.densetensor import DenseTensor
from tinygrad.sparsetensor import SparseTensor
import numpy as np
import tinygrad.optim as optim
from extra.utils import get_parameters


%load_ext autoreload
%autoreload 2

DEVICE:GPU


In [2]:
dim1 = 32
dim2 = 128
dim3 = 16

np.random.seed(9)

seedmat = np.random.randn(dim2,dim3).astype(np.float32) / 100

In [3]:
x_init = np.random.randn(dim1,dim2).astype(np.float32)
x = DenseTensor(x_init)
y = DenseTensor(np.random.randn(dim1,dim3))

warning, (32, 16) isn't float32, it's float64


In [4]:
w = SparseTensor.uniform(3,6)
DenseTensor(w.data).cpu().data.reshape(3,6)

ellw: 6 (3, 6)


array([[0.00900881, 0.00599214, 0.00755513, 0.00194298, 0.00168681,
        0.00714441],
       [0.00317871, 0.00603464, 0.00936129, 0.00616101, 0.00283417,
        0.00580005],
       [0.00593372, 0.00625892, 0.00946085, 0.00730415, 0.00452044,
        0.00706523]], dtype=float32)

In [5]:
class MLP:
  def __init__(self):
    self.W = DenseTensor(seedmat)

  def parameters(self):
    return get_parameters(self)

  def forward(self, x):
    out = x.dot(self.W)
    out = out.softmax()
    #out = out.mul(self.m).add(self.m).sum()
    return out

In [6]:
class MLP2:
  def __init__(self):
    self.W = SparseTensor(seedmat)
    #self.W = SparseTensor.uniform(dim2,dim3)

  def parameters(self):
    return get_parameters(self)

  def forward(self, x):
    out = x.dot(self.W)
    out = out.softmax()
    #out = out.mul(self.m).add(self.m).sum()
    return out

In [7]:
def loss_fn(y, y_pred):
    return ((y-y_pred)**2)**.5

### Dense

In [37]:
model = MLP()

In [38]:
iters = 2
optimizer = optim.SGD(model.parameters(), lr=.0001)

In [39]:
for i in range(iters):
    res = model.forward(x)
    optimizer.zero_grad()
    loss = loss_fn(res, y)
    loss.backward()
    optimizer.step()

In [40]:
model.parameters()

[<DenseTensor <GPUBuffer with shape (128, 16)> with grad <GPUBuffer with shape (128, 16)>>,
 <DenseTensor <GPUBuffer with shape (128, 16)> with grad None>]

In [41]:
res.cpu().data

array([[0.05573818, 0.05633433, 0.06563501, 0.06207809, 0.06393719,
        0.06361352, 0.0620168 , 0.06987084, 0.06801583, 0.06235965,
        0.05706918, 0.07404546, 0.06124875, 0.06210014, 0.05866341,
        0.05727357],
       [0.0540512 , 0.0555554 , 0.07173689, 0.05937867, 0.05573982,
        0.05944567, 0.0647753 , 0.06991503, 0.07263687, 0.0694721 ,
        0.05338153, 0.05521057, 0.06229831, 0.06633474, 0.06241272,
        0.06765521],
       [0.06542949, 0.06771529, 0.06045797, 0.05749483, 0.0583469 ,
        0.05674815, 0.05969064, 0.06446082, 0.06462204, 0.05766558,
        0.05934939, 0.06150933, 0.07207784, 0.07424898, 0.06239116,
        0.05779152],
       [0.06919651, 0.06115235, 0.06000684, 0.06546201, 0.06814209,
        0.06127458, 0.04603544, 0.05782926, 0.05662565, 0.05673028,
        0.06348448, 0.0678973 , 0.06351411, 0.0634392 , 0.07547157,
        0.06373829],
       [0.06738964, 0.06173735, 0.06126193, 0.05611748, 0.0680693 ,
        0.06301324, 0.05228309, 

In [42]:
loss.cpu().data

array([[2.09019661e+00, 2.24835706e+00, 7.83815980e-01, 5.73801875e-01,
        1.77110538e-01, 5.65624237e-01, 2.98013538e-01, 1.41246188e+00,
        7.20903158e-01, 4.60866123e-01, 6.98785365e-01, 8.98379147e-01,
        1.08615112e+00, 1.11011967e-01, 9.70433056e-02, 8.01649213e-01],
       [2.25519490e+00, 1.73285174e+00, 1.90730965e+00, 9.66127157e-01,
        1.54131186e+00, 3.67388465e-02, 1.71340823e-01, 1.10026670e+00,
        1.47404075e-01, 5.06499767e-01, 4.19561118e-01, 1.12750852e+00,
        6.77055418e-01, 2.83108997e+00, 2.86038667e-01, 6.93376243e-01],
       [1.11052060e+00, 9.29948092e-01, 1.35036695e+00, 2.96222746e-01,
        1.01726778e-01, 1.47533154e+00, 1.41719818e+00, 6.28078043e-01,
        4.70870808e-02, 4.97990787e-01, 2.10148597e+00, 7.85633147e-01,
        9.48857218e-02, 1.25674471e-01, 9.00201276e-02, 1.29294321e-02],
       [1.23583710e+00, 7.89709628e-01, 5.56398571e-01, 4.02661860e-01,
        2.22038239e-01, 2.05190077e-01, 6.84774816e-01, 1.045

In [43]:
res.grad.cpu().data

array([[-1.        ,  0.99999994,  1.        , -1.        , -1.        ,
        -1.        ,  1.        , -0.99999994, -1.0000001 , -1.        ,
        -1.        , -1.0000001 ,  1.        ,  1.        ,  1.        ,
         1.        ],
       [-0.99999994,  1.        , -1.        ,  0.99999994, -0.99999994,
        -0.99999994,  1.        ,  1.        ,  0.9999999 ,  0.99999994,
        -0.99999994,  0.99999994,  0.99999994,  1.        ,  1.        ,
        -1.        ],
       [ 1.        , -1.        , -1.        ,  0.99999994, -1.        ,
        -1.        ,  1.        , -0.99999994,  1.        ,  1.        ,
         1.        , -0.99999994, -1.        , -0.9999998 ,  1.        ,
         0.9999998 ],
       [ 0.99999994,  1.        , -1.        ,  1.0000001 ,  1.0000001 ,
         0.9999999 , -1.0000001 , -1.        ,  1.        , -1.        ,
        -1.        ,  0.99999994, -1.        ,  1.0000001 ,  1.        ,
        -1.        ],
       [-1.        , -1.        , -1

In [44]:
model.W.grad.cpu().data

array([[-0.01799509,  0.0867635 , -0.49340323, ...,  0.5353738 ,
         0.30732223,  0.07724259],
       [-0.17582685, -0.02344363, -0.34583828, ...,  0.4913508 ,
         0.04254012,  0.04201175],
       [ 0.03357629,  0.40655234,  0.4266157 , ..., -0.10753001,
        -0.19968747, -0.0144543 ],
       ...,
       [ 0.14695   , -0.43643492,  0.00249395, ..., -0.17832682,
         0.48194864,  0.23352517],
       [ 0.12411024,  0.16000795, -0.39656764, ...,  0.08061278,
         0.5410387 , -0.2249679 ],
       [ 0.39904255, -0.31726706, -0.160201  , ..., -0.34447545,
         0.21502733,  0.04214117]], dtype=float32)

### Second

In [45]:
model2 = MLP2()

In [46]:
optimizer2 = optim.SGD(model2.parameters(), lr=.0001)

In [47]:
for i in range(iters):
    res2 = model2.forward(x)
    optimizer2.zero_grad()
    loss2 = loss_fn(res2, y)
    loss2.backward()
    optimizer2.step()

In [48]:
res2.cpu().data==res.cpu().data

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

In [49]:
res2.cpu().data

array([[0.05573818, 0.05633433, 0.06563501, 0.06207809, 0.06393719,
        0.06361352, 0.0620168 , 0.06987084, 0.06801583, 0.06235965,
        0.05706918, 0.07404546, 0.06124875, 0.06210014, 0.05866341,
        0.05727357],
       [0.0540512 , 0.0555554 , 0.07173689, 0.05937867, 0.05573982,
        0.05944567, 0.0647753 , 0.06991503, 0.07263687, 0.0694721 ,
        0.05338153, 0.05521057, 0.06229831, 0.06633474, 0.06241272,
        0.06765521],
       [0.06542949, 0.06771529, 0.06045797, 0.05749483, 0.0583469 ,
        0.05674815, 0.05969064, 0.06446082, 0.06462204, 0.05766558,
        0.05934939, 0.06150933, 0.07207784, 0.07424898, 0.06239116,
        0.05779152],
       [0.06919651, 0.06115235, 0.06000684, 0.06546201, 0.06814209,
        0.06127458, 0.04603544, 0.05782926, 0.05662565, 0.05673028,
        0.06348448, 0.0678973 , 0.06351411, 0.0634392 , 0.07547157,
        0.06373829],
       [0.06738964, 0.06173735, 0.06126192, 0.05611748, 0.0680693 ,
        0.06301324, 0.05228309, 

In [50]:
loss.cpu().data==loss2.cpu().data

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

In [51]:
loss2.cpu().data

array([[2.09019661e+00, 2.24835706e+00, 7.83815980e-01, 5.73801875e-01,
        1.77110538e-01, 5.65624237e-01, 2.98013538e-01, 1.41246188e+00,
        7.20903158e-01, 4.60866123e-01, 6.98785365e-01, 8.98379147e-01,
        1.08615112e+00, 1.11011967e-01, 9.70433056e-02, 8.01649213e-01],
       [2.25519490e+00, 1.73285174e+00, 1.90730965e+00, 9.66127157e-01,
        1.54131186e+00, 3.67388465e-02, 1.71340823e-01, 1.10026670e+00,
        1.47404075e-01, 5.06499767e-01, 4.19561118e-01, 1.12750852e+00,
        6.77055418e-01, 2.83108997e+00, 2.86038667e-01, 6.93376243e-01],
       [1.11052060e+00, 9.29948092e-01, 1.35036695e+00, 2.96222746e-01,
        1.01726778e-01, 1.47533154e+00, 1.41719818e+00, 6.28078043e-01,
        4.70870808e-02, 4.97990787e-01, 2.10148597e+00, 7.85633147e-01,
        9.48857218e-02, 1.25674471e-01, 9.00201276e-02, 1.29294321e-02],
       [1.23583710e+00, 7.89709628e-01, 5.56398571e-01, 4.02661860e-01,
        2.22038239e-01, 2.05190077e-01, 6.84774816e-01, 1.045

In [52]:
res.grad.cpu().data==res2.grad.cpu().data

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  Tru

In [53]:
res2.grad.cpu().data

array([[-1.        ,  0.99999994,  1.        , -1.        , -1.        ,
        -1.        ,  1.        , -0.99999994, -1.0000001 , -1.        ,
        -1.        , -1.0000001 ,  1.        ,  1.        ,  1.        ,
         1.        ],
       [-0.99999994,  1.        , -1.        ,  0.99999994, -0.99999994,
        -0.99999994,  1.        ,  1.        ,  0.9999999 ,  0.99999994,
        -0.99999994,  0.99999994,  0.99999994,  1.        ,  1.        ,
        -1.        ],
       [ 1.        , -1.        , -1.        ,  0.99999994, -1.        ,
        -1.        ,  1.        , -0.99999994,  1.        ,  1.        ,
         1.        , -0.99999994, -1.        , -0.9999998 ,  1.        ,
         0.9999998 ],
       [ 0.99999994,  1.        , -1.        ,  1.0000001 ,  1.0000001 ,
         0.9999999 , -1.0000001 , -1.        ,  1.        , -1.        ,
        -1.        ,  0.99999994, -1.        ,  1.0000001 ,  1.        ,
        -1.        ],
       [-1.        , -1.        , -1

In [54]:
model.W.grad, model2.W.grad

(<DenseTensor <GPUBuffer with shape (128, 16)> with grad None>,
 <DenseTensor <GPUBuffer with shape (128, 16)> with grad None>)

In [55]:
model.W.grad.cpu().data==model2.W.grad.cpu().data

array([[False, False, False, ...,  True,  True,  True],
       [ True, False, False, ...,  True,  True,  True],
       [False,  True, False, ...,  True, False,  True],
       ...,
       [ True, False, False, ..., False,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True, False, False, ...,  True,  True, False]])

In [56]:
model.W.grad.cpu().data

array([[-0.01799509,  0.0867635 , -0.49340323, ...,  0.5353738 ,
         0.30732223,  0.07724259],
       [-0.17582685, -0.02344363, -0.34583828, ...,  0.4913508 ,
         0.04254012,  0.04201175],
       [ 0.03357629,  0.40655234,  0.4266157 , ..., -0.10753001,
        -0.19968747, -0.0144543 ],
       ...,
       [ 0.14695   , -0.43643492,  0.00249395, ..., -0.17832682,
         0.48194864,  0.23352517],
       [ 0.12411024,  0.16000795, -0.39656764, ...,  0.08061278,
         0.5410387 , -0.2249679 ],
       [ 0.39904255, -0.31726706, -0.160201  , ..., -0.34447545,
         0.21502733,  0.04214117]], dtype=float32)

In [57]:
model2.W.grad.cpu().data

array([[-0.0179951 ,  0.08676344, -0.493403  , ...,  0.5353738 ,
         0.30732223,  0.07724259],
       [-0.17582685, -0.02344361, -0.34583834, ...,  0.4913508 ,
         0.04254012,  0.04201175],
       [ 0.03357628,  0.40655234,  0.42661572, ..., -0.10753001,
        -0.19968748, -0.0144543 ],
       ...,
       [ 0.14695   , -0.4364349 ,  0.00249383, ..., -0.1783268 ,
         0.48194864,  0.23352517],
       [ 0.12411024,  0.16000795, -0.39656764, ...,  0.08061278,
         0.5410387 , -0.2249679 ],
       [ 0.39904255, -0.31726703, -0.16020109, ..., -0.34447545,
         0.21502733,  0.04214117]], dtype=float32)

In [58]:
model.W.grad

<DenseTensor <GPUBuffer with shape (128, 16)> with grad None>

In [59]:
model2.W.to_numpy()

nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newco

newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
ne

array([[ 1.46889979e-05, -2.91277375e-03, -1.10620027e-02, ...,
         6.96795201e-03,  1.81667004e-02,  4.29224316e-03],
       [ 1.54624498e-02, -9.00251884e-03, -1.30208221e-03, ...,
        -2.10160315e-02, -8.34368286e-04,  2.45434176e-02],
       [-2.67278161e-02, -9.21409577e-03, -2.35848571e-03, ...,
        -9.20911971e-03,  1.92124397e-02, -1.50014809e-03],
       ...,
       [-8.76534265e-03, -9.62305523e-04, -1.00406595e-02, ...,
         1.53807933e-02,  1.06349157e-03, -2.57564220e-03],
       [-6.94472063e-03,  4.72947722e-03,  5.60669415e-03, ...,
         1.28799407e-02,  1.61947627e-02, -2.24846276e-03],
       [ 2.11743917e-02,  1.42116463e-02,  4.21043718e-03, ...,
         7.45451497e-03,  9.59640276e-03, -7.09455879e-03]])

In [60]:
denserec = model.W.cpu().data 
denserec

array([[ 1.4688996e-05, -2.9127738e-03, -1.1062003e-02, ...,
         6.9679520e-03,  1.8166700e-02,  4.2922432e-03],
       [ 1.5462450e-02, -9.0025188e-03, -1.3020822e-03, ...,
        -2.1016032e-02, -8.3436829e-04,  2.4543418e-02],
       [-2.6727816e-02, -9.2140958e-03, -2.3584857e-03, ...,
        -9.2091197e-03,  1.9212440e-02, -1.5001481e-03],
       ...,
       [-8.7653426e-03, -9.6230546e-04, -1.0040659e-02, ...,
         1.5380793e-02,  1.0634916e-03, -2.5756422e-03],
       [-6.9447206e-03,  4.7294772e-03,  5.6066941e-03, ...,
         1.2879941e-02,  1.6194763e-02, -2.2484628e-03],
       [ 2.1174392e-02,  1.4211646e-02,  4.2104372e-03, ...,
         7.4545150e-03,  9.5964028e-03, -7.0945588e-03]], dtype=float32)

In [61]:
sparserec = DenseTensor(model2.W.data).cpu().data.reshape(denserec.shape)
sparserec

array([[ 1.4688998e-05, -2.9127738e-03, -1.1062003e-02, ...,
         6.9679520e-03,  1.8166700e-02,  4.2922432e-03],
       [ 1.5462450e-02, -9.0025188e-03, -1.3020822e-03, ...,
        -2.1016032e-02, -8.3436829e-04,  2.4543418e-02],
       [-2.6727816e-02, -9.2140958e-03, -2.3584857e-03, ...,
        -9.2091197e-03,  1.9212440e-02, -1.5001481e-03],
       ...,
       [-8.7653426e-03, -9.6230552e-04, -1.0040659e-02, ...,
         1.5380793e-02,  1.0634916e-03, -2.5756422e-03],
       [-6.9447206e-03,  4.7294772e-03,  5.6066941e-03, ...,
         1.2879941e-02,  1.6194763e-02, -2.2484628e-03],
       [ 2.1174392e-02,  1.4211646e-02,  4.2104372e-03, ...,
         7.4545150e-03,  9.5964028e-03, -7.0945588e-03]], dtype=float32)

In [62]:
denserec - sparserec

array([[-1.8189894e-12,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 0.0000000e+00,  5.8207661e-11,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [63]:
sparserec2 = DenseTensor(model2.W.datat).cpu().data.reshape(denserec.shape[1], -1)
sparserec2.T

array([[ 1.4688998e-05, -2.9127738e-03, -1.1062003e-02, ...,
         6.9679520e-03,  1.8166700e-02,  4.2922432e-03],
       [ 1.5462450e-02, -9.0025188e-03, -1.3020822e-03, ...,
        -2.1016032e-02, -8.3436829e-04,  2.4543418e-02],
       [-2.6727816e-02, -9.2140958e-03, -2.3584857e-03, ...,
        -9.2091197e-03,  1.9212440e-02, -1.5001481e-03],
       ...,
       [-8.7653426e-03, -9.6230552e-04, -1.0040659e-02, ...,
         1.5380793e-02,  1.0634916e-03, -2.5756422e-03],
       [-6.9447206e-03,  4.7294772e-03,  5.6066941e-03, ...,
         1.2879941e-02,  1.6194763e-02, -2.2484628e-03],
       [ 2.1174392e-02,  1.4211646e-02,  4.2104372e-03, ...,
         7.4545150e-03,  9.5964028e-03, -7.0945588e-03]], dtype=float32)

In [64]:
denserec - sparserec2.T

array([[-1.8189894e-12,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 0.0000000e+00,  5.8207661e-11,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [65]:
seedmat - denserec

array([[-3.6034498e-06,  1.7333077e-05, -9.8660588e-05, ...,
         1.0708487e-04,  6.1457977e-05,  1.5446916e-05],
       [-3.5153702e-05, -4.6929345e-06, -6.9167931e-05, ...,
         9.8263845e-05,  8.4908679e-06,  8.4079802e-06],
       [ 6.7148358e-06,  8.1302598e-05,  8.5342210e-05, ...,
        -2.1496788e-05, -3.9936975e-05, -2.8803479e-06],
       ...,
       [ 2.9395334e-05, -8.7261433e-05,  5.0663948e-07, ...,
        -3.5696663e-05,  9.6408068e-05,  4.6687666e-05],
       [ 2.4823938e-05,  3.1979755e-05, -7.9311430e-05, ...,
         1.6121194e-05,  1.0822527e-04, -4.5036897e-05],
       [ 7.9823658e-05, -6.3443556e-05, -3.2059848e-05, ...,
        -6.8909489e-05,  4.3023378e-05,  8.3725899e-06]], dtype=float32)

In [68]:
m2dense = model2.W.to_numpy()

nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newco

newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
nnzs: 16
newcol: 0
newcol: 1
newcol: 2
newcol: 3
newcol: 4
newcol: 5
newcol: 6
newcol: 7
newcol: 8
newcol: 9
newcol: 10
newcol: 11
newcol: 12
newcol: 13
newcol: 14
newcol: 15
